In [0]:
import os
import json
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DateType, StringType, TimestampType, IntegerType, FloatType

In [0]:
storageAccountName = 'equitymarketblob'
storageAccountAccessKey = '0nboKYno6oyL7Op44qYarGAfeq1EmdnrBUHWvYUyfDfIdGF990OzOw8KsL7RdEofxLC7W0WaHB66+AStrlJdZA=='
ContainerName = 'bcontainer1'

if not any(mount.mountPoint == '/mnt/blob1/' for mount in dbutils.fs.mounts()):
    try:
        dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(ContainerName, storageAccountName),
        mount_point = "/mnt/blob1",
        extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        )
    except Exception as e:
        print('mount exception')
display(dbutils.fs.ls('mnt/blob1'))

mount exception

path,name,size,modificationTime
dbfs:/mnt/blob1/data/,data/,0,0


In [0]:
# define a common schema 
schema = StructType([
    StructField('trade_dt', DateType(), True),
    StructField('rec_type', StringType(),True),
    StructField('symbol', StringType(),True),
    StructField('exchange', StringType(),True),
    StructField('event_tm', TimestampType(),True),
    StructField('event_seq_nb', IntegerType(),True),
    StructField('arrival_tm', TimestampType(),True),
    StructField('trade_pr', FloatType(),True),
    StructField('bid_pr', FloatType(),True),
    StructField('bid_size', IntegerType(),True),
    StructField('ask_pr', FloatType(),True),
    StructField('ask_size', IntegerType(),True),
    StructField('partition', StringType(),True)])

In [0]:
#parse csv file
def parse_csv(line:str):
    record =line.split(',')
    try:
        if record[2]=='Q':
            return (datetime.strptime(record[0], '%Y-%m-%d'),
                    record[2],
                    record[3],
                    record[6],
                    datetime.strptime(record[4], '%Y-%m-%d %H:%M:%S.%f'),
                    int(record[5]),
                    datetime.strptime(record[1], '%Y-%m-%d %H:%M:%S.%f'),
                    None,
                    float(record[7]),
                    int(record[8]),
                    float(record[9]),
                    int(record[10]),
                    'Q')
        elif record[2]=='T':
             return (datetime.strptime(record[0], '%Y-%m-%d'),
                    record[2],
                    record[3],
                    record[6],
                    datetime.strptime(record[4], '%Y-%m-%d %H:%M:%S.%f'),
                    int(record[5]),
                    datetime.strptime(record[1], '%Y-%m-%d %H:%M:%S.%f'),
                    float(record[7]),
                    None,
                    int(record[8]),
                    None,
                    None,
                    'T')
    except Exception as e:   
            print("parse_csv exception:"+line)
            return (None,None,None,None,None,None,None,None,None,None,None,None,"B")

In [0]:
#parse json file
def parse_json(line:str):
    record=json.loads(line)
    try:
        # [logic to parse records]
        if record['event_type']=='Q':
            #if any of the fields missing, then, return a "B" record
            #if all(i in record for i in ['trade_dt', 'event_type', 'symbol', 'exchange', 'event_tm',
            #   'event_seq_nb', 'file_tm', 'bid_pr', 'bid_size', 'ask_pr', 'ask_size']):
            return (datetime.strptime(record['trade_dt'], '%Y-%m-%d'), 
                    record['event_type'], 
                    record['symbol'], 
                    record['exchange'], 
                    datetime.strptime(field['event_tm'], '%Y-%m-%d %H:%M:%S.%f'),
                    int(record['event_seq_nb']), 
                    datetime.strptime(field['file_tm'], '%Y-%m-%d %H:%M:%S.%f'), 
                    None, 
                    float(record['bid_pr']), 
                    int(record['bid_size']), 
                    float(record['ask_pr']), 
                    int(record['ask_size']),
                    "Q")
            #else:
                #return (None,None,None,None,None,None,None,None,None,None,None,None,"B")
        elif record['event_type']=='T':
            #if any of the fields missing, then, return a "B" record
            #if all(i in record for i in ['execution_id', trade_dt', 'event_type', 'symbol', 'exchange',
            #  'event_tm', 'event_seq_nb', 'file_tm', 'price', 'size']):
            return (datetime.strptime(field['trade_dt'], '%Y-%m-%d'), 
                    record['event_type'], 
                    record['symbol'], 
                    record['exchange'], 
                    datetime.strptime(record['event_tm'], '%Y-%m-%d %H:%M:%S.%f'),
                    int(record['event_seq_nb']), 
                    datetime.strptime(record['file_tm'], '%Y-%m-%d %H:%M:%S.%f'),
                    float(record['price']), 
                    None, 
                    int(record['size']), 
                    None, 
                    None,
                    "T")
            #else:
                #return (None,None,None,None,None,None,None,None,None,None,None,None,"B")
    except Exception as e:
            print("parse_json exception"+line)
            return (None,None,None,None,None,None,None,None,None,None,None,None,"B")

In [0]:
#rdd for csv file of date 0805
#path = 'C:/sb/equity-market-data-analysis/data/csv/2020-08-05/NYSE/'
path = '/mnt/blob1/data/csv/2020-08-05/NYSE/'
name =os.listdir('/dbfs'+path)
print (name)
raw_csv=spark.sparkContext.textFile("{}/{}".format(path, name[3]))
raw_csv.count()
csv0805_rdd=raw_csv.map(parse_csv)

['._SUCCESS.crc', '.part-00000-5e4ced0a-66e2-442a-b020-347d0df4df8f-c000.txt.crc', '_SUCCESS', 'part-00000-5e4ced0a-66e2-442a-b020-347d0df4df8f-c000.txt']

In [0]:
#rdd for csv file of date 0806
#path = 'C:/sb/equity-market-data-analysis/data/csv/2020-08-06/NYSE/'
path = '/mnt/blob1/data/csv/2020-08-06/NYSE/'
name =os.listdir('/dbfs'+path)
raw_csv=spark.sparkContext.textFile(path+name[3])
csv0806_rdd=raw_csv.map(parse_csv)

In [0]:
#rdd for json file of date 0805
#path = 'C:/sb/equity-market-data-analysis/data/json/2020-08-05/NASDAQ/'
path = '/mnt/blob1/data/json/2020-08-05/NASDAQ/'
name =os.listdir('/dbfs'+path)
raw_json=spark.sparkContext.textFile(path+name[3])
json0805_rdd=raw_json.map(parse_json)

In [0]:
#rdd for json file of date 0806
#path = 'C:/sb/equity-market-data-analysis/data/json/2020-08-06/NASDAQ/'
path = '/mnt/blob1/data/json/2020-08-06/NASDAQ/'
name =os.listdir('/dbfs'+path)
raw_json=spark.sparkContext.textFile(path+name[3])
json0806_rdd=raw_json.map(parse_json)

In [0]:
#union the DASDAQ and NYSE to get the rdd for date 0805
rdd0805 = csv0805_rdd.union(json0805_rdd)
rdd0806 = csv0806_rdd.union(json0806_rdd)

In [0]:
df = spark.createDataFrame(rdd0805, schema=schema)
df.show()
df.write.partitionBy("partition").mode("overwrite").parquet("output_dir/2020-08-05")
df = spark.createDataFrame(rdd0806, schema=schema)
df.show()
df.write.partitionBy("partition").mode("overwrite").parquet("output_dir/2020-08-06")

+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
 trade_dt|rec_type|symbol|exchange| event_tm|event_seq_nb| arrival_tm| trade_pr| bid_pr|bid_size| ask_pr|ask_size|partition|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:34:...| 1|2020-08-05 09:30:00| null| 75.30255| 100| 75.35917| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:40:...| 2|2020-08-05 09:30:00| null| 77.20875| 100| 78.90918| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:50:...| 3|2020-08-05 09:30:00| null| 77.15973| 100| 77.33205| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 09:57:...| 4|2020-08-05 09:30:00| null|79.299774| 100| 80.08399| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:06:...| 5|2020-08-05 09:30:00| null|77.863495| 100| 78.30821| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:11:...| 6|2020-08-05 09:30:00| null| 78.74799| 100|80.329056| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:17:...| 7|2020-08-05 09:30:00| null| 77.16837| 100| 77.97598| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:23:...| 8|2020-08-05 09:30:00| null| 78.06551| 100| 78.80516| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:31:...| 9|2020-08-05 09:30:00| null| 76.69653| 100| 78.56943| 100| Q|
2020-08-05| T| SYMA| NYSE|2020-08-05 10:37:...| 10|2020-08-05 09:30:00|79.194885| null| 912| null| null| T|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:45:...| 11|2020-08-05 09:30:00| null| 75.10474| 100|77.096565| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 10:54:...| 12|2020-08-05 09:30:00| null| 74.52148| 100| 75.66917| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:03:...| 13|2020-08-05 09:30:00| null| 75.16876| 100| 76.70698| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:12:...| 14|2020-08-05 09:30:00| null| 74.56633| 100|74.621376| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:19:...| 15|2020-08-05 09:30:00| null| 74.49803| 100|76.475914| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:27:...| 16|2020-08-05 09:30:00| null| 75.22889| 100|76.143684| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:35:...| 17|2020-08-05 09:30:00| null| 76.62792| 100| 76.99149| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:42:...| 18|2020-08-05 09:30:00| null|77.094666| 100|78.159676| 100| Q|
2020-08-05| Q| SYMA| NYSE|2020-08-05 11:51:...| 19|2020-08-05 09:30:00| null|78.335365| 100| 79.39139| 100| Q|
2020-08-05| T| SYMA| NYSE|2020-08-05 11:56:...| 20|2020-08-05 09:30:00| 76.49093| null| 962| null| null| T|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
only showing top 20 rows

+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
 trade_dt|rec_type|symbol|exchange| event_tm|event_seq_nb| arrival_tm| trade_pr| bid_pr|bid_size| ask_pr|ask_size|partition|
+----------+--------+------+--------+--------------------+------------+-------------------+---------+---------+--------+---------+--------+---------+
2020-08-06| Q| SYMA| NYSE|2020-08-06 09:39:...| 1|2020-08-06 09:30:00| null| 77.67913| 100|78.437355| 100| Q|
2020-08-06| Q| SYMA| NYSE|2020-08-06 09:47:...| 2|2020-08-06 09:30:00| null| 76.53373| 100| 76.94425| 100| Q|
2020-08-06| Q| SYMA| NYSE|2020-08-06 09:56:...| 3|2020-08-06 09:30:00| null|75.120605| 100| 75.39408| 100| Q|
2020-08-06| Q| SYMA| NYSE|2020-08-06 10:03:...| 4|2020-08-06 09:30:00| null| 74.86369| 100| 75.76861| 100| Q|
2020-08-06| Q| SYMA| NYSE|2020-08-06 10:09:...| 5|2020-08-06 09:30:00| null| 77.7765| 100|78.801094| 100| Q|
2020-08-06| Q| SYMA| NYSE|2020-08-06 10:18:...| 6|2020-08-06 09:30:00| null| 75.02947| 100|75.690575| 100| Q|
2020-08-06| Q| SYMA| NYSE|2020-08-06 10:25:...| 7|2020-08-06 09:30:00| null| 75.36016| 100| 75.46482| 100| Q|


In [0]:
dbutils.notebook.exit('SUCCESS')